In [2]:
import sys
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
  
ps = PorterStemmer()
sys.path.append("..")
from rcn_py import orcid

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


A simple workflow

In [3]:
# query ORCID by a fullname
orcid_id = orcid.name_to_orcid_id('Peter Kalverla')
orcid_id

'0000-0002-5025-7862'

In [4]:
fullname = orcid.from_orcid_to_name(orcid_id)
fullname

'Peter Kalverla'

In [5]:
# query ORCID for an ORCID record
orcid_record = orcid.query_orcid_for_record(orcid_id)
orcid_record

{'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-5025-7862',
  'path': '0000-0002-5025-7862',
  'host': 'orcid.org'},
 'preferences': {'locale': 'en'},
 'history': {'creation-method': 'DIRECT',
  'completion-date': None,
  'submission-date': {'value': 1440847595525},
  'last-modified-date': {'value': 1677858209460},
  'claimed': True,
  'source': None,
  'deactivation-date': None,
  'verified-email': True,
  'verified-primary-email': True},
 'person': {'last-modified-date': {'value': 1614867500639},
  'name': {'created-date': {'value': 1460759791637},
   'last-modified-date': {'value': 1460759791637},
   'given-names': {'value': 'Peter'},
   'family-name': {'value': 'Kalverla'},
   'credit-name': None,
   'source': None,
   'visibility': 'public',
   'path': '0000-0002-5025-7862'},
  'other-names': {'last-modified-date': None,
   'other-name': [],
   'path': '/0000-0002-5025-7862/other-names'},
  'biography': None,
  'researcher-urls': {'last-modified-date': None,
   'research

In [6]:
docs = orcid.extract_works_section(orcid_record)
docs

[{'last-modified-date': {'value': 1654040356185},
  'external-ids': {'external-id': [{'external-id-type': 'doi',
     'external-id-value': '10.5194/egusphere-egu21-9387',
     'external-id-normalized': {'value': '10.5194/egusphere-egu21-9387',
      'transient': True},
     'external-id-normalized-error': None,
     'external-id-url': {'value': 'https://doi.org/10.5194/egusphere-egu21-9387'},
     'external-id-relationship': 'self'}]},
  'work-summary': [{'put-code': 89903286,
    'created-date': {'value': 1614838570891},
    'last-modified-date': {'value': 1654040356185},
    'source': {'source-orcid': None,
     'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
      'path': '0000-0001-9884-1913',
      'host': 'orcid.org'},
     'source-name': {'value': 'Crossref'},
     'assertion-origin-orcid': None,
     'assertion-origin-client-id': None,
     'assertion-origin-name': None},
    'title': {'title': {'value': 'A multi-model ensemble weighting method (Clim

In [15]:
# extract title and DOI
doi_list = []
title_list = []
for i in docs:
    doi, title = orcid.extract_doi(i)
    doi_list.append(doi)
    title_list.append(title)
print(doi_list)
print(title_list)

['10.5194/egusphere-egu21-9387', '10.5194/egusphere-egu21-6051', '10.5194/egusphere-egu21-7797', '10.5194/egusphere-egu21-4805', '10.5194/egusphere-egu21-3476', '10.5194/egusphere-egu21-4895', '10.5194/wes-5-1097-2020', '10.5194/egusphere-egu2020-21619', '10.18174/498797', '10.5194/wes-4-193-2019', '10.5194/wes-2018-79-supplement', '10.1002/we.2267']
['A multi-model ensemble weighting method (ClimWIP) in ESMValTool', 'Preprocessing of hydrological models&#8217; input in eWaterCycle with ESMValTool', 'Towards Open and FAIR Hydrological Modelling with eWaterCycle', 'Bringing ESMValTool to the Jupyter Lab', 'Recent developments on the Earth System Model Evaluation Tool', 'era5cli: the command line interface to ERA5 data', 'Clustering wind profile shapes to estimate airborne wind energy production', 'era5cli: The command line tool to download ERA5 data', 'Characterisation of offshore winds for energy applications', 'Low-level jets over the North Sea based on ERA5 and observations: together

In [16]:
clusters = orcid.orcid_lda_cluster(doi_list)
clusters

{'10.5194/egusphere-egu21-9387': 0,
 '10.5194/egusphere-egu21-6051': 0,
 '10.5194/egusphere-egu21-7797': 0,
 '10.5194/egusphere-egu21-4805': 0,
 '10.5194/egusphere-egu21-3476': 0,
 '10.5194/egusphere-egu21-4895': 0,
 '10.5194/wes-5-1097-2020': 0,
 '10.5194/egusphere-egu2020-21619': 0,
 '10.18174/498797': 5,
 '10.5194/wes-4-193-2019': 0,
 '10.5194/wes-2018-79-supplement': 2,
 '10.1002/we.2267': 0}

In [17]:
df, links = orcid.orcid_get_coauthors('Peter Kalverla')

In [18]:
df

,orcid,name,group
0,0000-0002-3986-1268,Ruth Lorenz,0
1,0000-0001-5760-4524,Lukas Brunner,0
2,0000-0002-5025-7862,Peter Kalverla,0
3,0000-0002-5413-9038,Stef Smeets,0
4,0000-0002-8928-7831,Jaro Camphuijsen,0
5,0000-0001-9005-8940,Bouwe Andela,0
6,0000-0001-8407-6472,Fakhereh Alidoost,0
7,0000-0003-0157-4818,Jerom Aerts,0
8,0000-0002-7200-3353,Nick van De Giesen,0
9,0000-0001-8367-1333,Gijs van Den Oord,0


In [19]:
links

[('0000-0002-3986-1268', '0000-0001-5760-4524'),
 ('0000-0002-3986-1268', '0000-0002-5025-7862'),
 ('0000-0002-3986-1268', '0000-0002-5413-9038'),
 ('0000-0002-3986-1268', '0000-0002-8928-7831'),
 ('0000-0002-3986-1268', '0000-0001-9005-8940'),
 ('0000-0001-5760-4524', '0000-0002-5025-7862'),
 ('0000-0001-5760-4524', '0000-0002-5413-9038'),
 ('0000-0001-5760-4524', '0000-0002-8928-7831'),
 ('0000-0001-5760-4524', '0000-0001-9005-8940'),
 ('0000-0002-5025-7862', '0000-0002-5413-9038'),
 ('0000-0002-5025-7862', '0000-0002-8928-7831'),
 ('0000-0002-5025-7862', '0000-0001-9005-8940'),
 ('0000-0002-5413-9038', '0000-0002-8928-7831'),
 ('0000-0002-5413-9038', '0000-0001-9005-8940'),
 ('0000-0002-8928-7831', '0000-0001-9005-8940'),
 ('0000-0001-8407-6472', '0000-0003-0157-4818'),
 ('0000-0001-8407-6472', '0000-0001-9005-8940'),
 ('0000-0001-8407-6472', '0000-0002-8928-7831'),
 ('0000-0001-8407-6472', '0000-0002-7200-3353'),
 ('0000-0001-8407-6472', '0000-0001-8367-1333'),
 ('0000-0001-8407-64

In [6]:
orcid.orcid_network("P_Kalverla", df, links)

In [34]:
from crossref.restful import Works
import gensim
import numpy as np
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()

In [32]:
def lemmatize_stemming(text):
    return ps.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [35]:
string = "a hh dfhgf iekshj &*O (djahj daj wad d."
preprocess(string)

['dfhgf', 'iekshj', 'djahj']

In [41]:
def orcid_lda_cluster(dois):
    
    cleaned_abs_corpus = []
    clusters = {}
    works = Works()
    for i in dois:
            w = works.doi(i)
            if 'abstract' in w.keys():
                cleaned_abs_corpus.append(preprocess(w['abstract']))
            elif 'title' in w.keys() and w['title']:
                cleaned_abs_corpus.append(preprocess(w['title'][0]))
            else:
                cleaned_abs_corpus.append([])

    dictionary = gensim.corpora.Dictionary(cleaned_abs_corpus) 
    corpus = [dictionary.doc2bow(text) for text in cleaned_abs_corpus]

    # np.random.seed(1)

    # lda_model = gensim.models.LdaMulticore(corpus=corpus,
    #                                        id2word=dictionary,
    #                                        num_topics=num_topics,
    #                                        chunksize= 4000, 
    #                                        batch= True,
    #                                        minimum_probability=0.001,
    #                                        iterations=350,
    #                                        passes=passes)

    lda_model =  gensim.models.LdaMulticore(corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)
    
    idx2topics = {}
    for idx, topic in lda_model.print_topics(-1):
        idx2topics[idx] = topic
 
    for i in range(len(cleaned_abs_corpus)):
        scores = []
        topics = []
        for j1,j2 in lda_model[corpus[i]]:
            topics.append(j1)
            scores.append(j2)
            clusters[dois[i]] = scores.index(max(scores))
        
    return clusters, idx2topics

In [42]:
orcid_lda_cluster(doi_list)

({'10.5194/egusphere-egu21-9387': 0,
  '10.5194/egusphere-egu21-6051': 0,
  '10.5194/egusphere-egu21-7797': 0,
  '10.5194/egusphere-egu21-4805': 0,
  '10.5194/egusphere-egu21-3476': 1,
  '10.5194/egusphere-egu21-4895': 0,
  '10.5194/wes-5-1097-2020': 0,
  '10.5194/egusphere-egu2020-21619': 0,
  '10.18174/498797': 5,
  '10.5194/wes-4-193-2019': 0,
  '10.5194/wes-2018-79-supplement': 4,
  '10.1002/we.2267': 6},
 {0: '0.002*"tool" + 0.002*"model" + 0.002*"data" + 0.002*"recip" + 0.002*"esmvaltool" + 0.002*"scientif" + 0.002*"process" + 0.002*"reproduc" + 0.002*"project" + 0.002*"jat"',
  1: '0.036*"avail" + 0.024*"download" + 0.024*"data" + 0.022*"model" + 0.020*"option" + 0.018*"command" + 0.016*"jat" + 0.016*"includ" + 0.014*"variabl" + 0.014*"autom"',
  2: '0.040*"tool" + 0.023*"recip" + 0.019*"diagnost" + 0.018*"user" + 0.017*"test" + 0.014*"commun" + 0.014*"improv" + 0.014*"scientif" + 0.014*"cmip" + 0.013*"evalu"',
  3: '0.002*"wind" + 0.002*"tool" + 0.002*"data" + 0.002*"model" + 0